# 테트리스 게임

### 1. 라이브러리 적재 및 초기화

* random: 난수 생성 라이브러리
* sys: 시스템 정보 라이브러리
* pygame: game을 만들기 위한 라이브러리
> pip install pygame [for cmd, terminal]
> conda install -c cogsci pygame [for conda environment]
> for the others, please contact me.

In [ ]:
from random import randrange as rand
import pygame, sys

# The configuration
config = {
	'cell_size':20,
	'cols':		16,
	'rows':		32,
	'delay':		750,
	'maxfps':	30
}

colors = [
(0,   0,   0  ),
(255, 0,   0  ),
(0,   150, 0  ),
(0,   0,   255),
(255, 120, 0  ),
(255, 255, 0  ),
(180, 0,   255),
(0,   220, 220)
]

# Define the shapes of the single parts
tetris_shapes = [
	[[1, 1, 1],
	 [0, 1, 0]],
	
	[[0, 2, 2],
	 [2, 2, 0]],
	
	[[3, 3, 0],
	 [0, 3, 3]],
	
	[[4, 0, 0],
	 [4, 4, 4]],
	
	[[0, 0, 5],
	 [5, 5, 5]],
	
	[[6, 6, 6, 6]],
	
	[[7, 7],
	 [7, 7]]
]

### 1. 기본 함수 정의

* rotate_clockwise(shape: List) 시계 방향으로 회전
> for ::elt:: in  ::container::<br /> : 반복문
> [... for ::elt:: in ::container::] : list comprehesion, 리스트를 만들어주는 문법

* check_collision(board, shape, offset: int) 충돌을 확인
> try ... catch ...  exception ...: 에러 처리 문법

* remove_row(board, row) : 한 줄이 다 차면 지워줌
> del: destruction(소멸자) 문법

* join_matrixes(mat1, mat2, mat2_off) 테트리스(메트릭스)를 합쳐줌

* new_board(): 새로운 테트리스를 만듦

In [ ]:
def rotate_clockwise(shape):
	return [[shape[y][x] for y in range(len(shape))]
		for x in range(len(shape[0]) - 1, -1, -1)]

def check_collision(board, shape, offset):
	off_x, off_y = offset
	for cy, row in enumerate(shape):
		for cx, cell in enumerate(row):
			try:
				if cell and board[ cy + off_y ][ cx + off_x ]:
					return True
			except IndexError:
				return True
	return False

def remove_row(board, row):
	del board[row]
	return [[0 for i in range(config['cols'])]] + board
	
def join_matrixes(mat1, mat2, mat2_off):
	off_x, off_y = mat2_off
	for cy, row in enumerate(mat2):
		for cx, val in enumerate(row):
			mat1[cy+off_y-1	][cx+off_x] += val
	return mat1

def new_board():
	board = [ [ 0 for x in range(config['cols']) ]
			for y in range(config['rows']) ]
	board += [[ 1 for x in range(config['cols'])]]
	return board


### 2. 테트리스 앱에 관한 함수와 변수를 묶어서 정의 (Class)

테트리스 앱은 다음과 같은 것들이 있어야 한다고 생각할 수 있다.
* 우선 실행이 되야하며
* 화면 크기가 존재하며
* 점수, 승리 및 패배 알릴 수 있어야 하며
* 테트리스를 움직일 수 있어야 하고,
* 한 줄이 차면 그 줄을 치워 줘야 한다.
* 또한 키보드의 입력을 받아 움직여야 한다.

* pygame: 게임에 대한 전반적인 것을 담고 있음. 예를 들어 화면, 음악, 실행 등에 관한 내용을 담아둔 engine 성격의 라이브러리.
> unity engine, unreal engine 생각해봐요.

* TetrisApp(Object): 클래스라는 method와 attribute의 묶음.
> self는 instance (현재 존재하는 객체)를 지칭한다.

* init(self): constructor (생성자) 함수, 파이썬은 여기서 클래스와 관련된 attribute(성질, 변수)들을 선언할 수 있다.
> 예를 들어 width, height는 화면의 크기에 관한 테트리스 게임의 성질이며, screen은 이 성질들을 받아 화면을 그려준다.

* new_stone(self): 새로운 테트리스를 화면 맨 위 중앙에 생성해줌. 생성할 때 맨 위까지 채워져있으면 테트리스 게임 집니다.

* init_game(self): 게임 시작!

* center_msg(self, msg): 파이썬 게임 중앙에 메세지 출력해줍니다.

* draw_matrix(self, matrix, offset): 테트리스 그려줌. pygame이라는 라이브러리가 화면을 구성하고 있으므로 여기에 네모 좀 그려달라고 요청(pygame.Rect()).

* move(self, delta_x): 양 옆으로 움직여 줌. 대신 양 쪽 옆 마지막 칸에 닿으면 못 움직이게 함.

* quit(self): 끝!

* drop(self): 한 칸 씩 내려가게 함. 그런데 매 칸 내려갈 때마다 줄이 다 차거나, 다른 블록이 있을 경우 이에 해당하는 행동을 함.
> while: 반복문.

* run(self): 전체적으로 실행하는 함수.

In [ ]:
class TetrisApp(object):
	def __init__(self):
		pygame.init()
		pygame.key.set_repeat(250,25)
		self.width = config['cell_size']*config['cols']
		self.height = config['cell_size']*config['rows']
		
		self.screen = pygame.display.set_mode((self.width, self.height))
		pygame.event.set_blocked(pygame.MOUSEMOTION)
		self.init_game()
	
	def new_stone(self):
		self.stone = tetris_shapes[rand(len(tetris_shapes))]
		self.stone_x = int(config['cols']/2 - len(self.stone[0])/2)
		self.stone_y = 0
		
		if check_collision(self.board,
		                   self.stone,
		                   (self.stone_x, self.stone_y)):
			self.gameover = True
	
	def init_game(self):
		self.board = new_board()
		self.new_stone()
	
	def center_msg(self, msg):
		for i, line in enumerate(msg.splitlines()):
			msg_image =  pygame.font.Font(
				pygame.font.get_default_font(), 12).render(
					line, False, (255,255,255), (0,0,0))
		
			msgim_center_x, msgim_center_y = msg_image.get_size()
			msgim_center_x //= 2
			msgim_center_y //= 2
		
			self.screen.blit(msg_image, (
			  self.width // 2-msgim_center_x,
			  self.height // 2-msgim_center_y+i*22))
	
	def draw_matrix(self, matrix, offset):
		off_x, off_y  = offset
		for y, row in enumerate(matrix):
			for x, val in enumerate(row):
				if val:
					pygame.draw.rect(
						self.screen,
						colors[val],
						pygame.Rect(
							(off_x+x) *
							  config['cell_size'],
							(off_y+y) *
							  config['cell_size'], 
							config['cell_size'],
							config['cell_size']),0)
	
	def move(self, delta_x):
		if not self.gameover and not self.paused:
			new_x = self.stone_x + delta_x
			if new_x < 0:
				new_x = 0
			if new_x > config['cols'] - len(self.stone[0]):
				new_x = config['cols'] - len(self.stone[0])
			if not check_collision(self.board,
			                       self.stone,
			                       (new_x, self.stone_y)):
				self.stone_x = new_x
	def quit(self):
		self.center_msg("Exiting...")
		pygame.display.update()
		sys.exit()
	
	def drop(self):
		if not self.gameover and not self.paused:
			self.stone_y += 1
			if check_collision(self.board,
			                   self.stone,
			                   (self.stone_x, self.stone_y)):
				self.board = join_matrixes(
				  self.board,
				  self.stone,
				  (self.stone_x, self.stone_y))
				self.new_stone()
				while True:
					for i, row in enumerate(self.board[:-1]):
						if 0 not in row:
							self.board = remove_row(
							  self.board, i)
							break
					else:
						break
	
	def rotate_stone(self):
		if not self.gameover and not self.paused:
			new_stone = rotate_clockwise(self.stone)
			if not check_collision(self.board,
			                       new_stone,
			                       (self.stone_x, self.stone_y)):
				self.stone = new_stone
	
	def toggle_pause(self):
		self.paused = not self.paused
	
	def start_game(self):
		if self.gameover:
			self.init_game()
			self.gameover = False
	
	def run(self):
		key_actions = {
			'ESCAPE':	self.quit,
			'LEFT':		lambda:self.move(-1),
			'RIGHT':	lambda:self.move(+1),
			'DOWN':		self.drop,
			'UP':		self.rotate_stone,
			'p':		self.toggle_pause,
			'SPACE':	self.start_game
		}
		
		self.gameover = False
		self.paused = False
		
		pygame.time.set_timer(pygame.USEREVENT+1, config['delay'])
		dont_burn_my_cpu = pygame.time.Clock()
		while 1:
			self.screen.fill((0,0,0))
			if self.gameover:
				self.center_msg("""Game Over! Press space to continue""")
			else:
				if self.paused:
					self.center_msg("Paused")
				else:
					self.draw_matrix(self.board, (0,0))
					self.draw_matrix(self.stone,
					                 (self.stone_x,
					                  self.stone_y))
			pygame.display.update()
			
			for event in pygame.event.get():
				if event.type == pygame.USEREVENT+1:
					self.drop()
				elif event.type == pygame.QUIT:
					self.quit()
				elif event.type == pygame.KEYDOWN:
					for key in key_actions:
						if event.key == eval("pygame.K_"+key):
							key_actions[key]()
					
			dont_burn_my_cpu.tick(config['maxfps'])

### 3. 실행가능한 파일로 설정

현재까지는 설정 단계이며, 실행되는 line이 없다.

> if \_\_name\_\_ == '\_\_main\_\_':


는 c에서의 main 함수와 비슷하다.


In [ ]:
if __name__ == '__main__':
	App = TetrisApp()
	App.run()